---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [1]:
import pandas as pd
import numpy as np

#def print_full(x):
#    pd.set_option('display.max_rows', len(x))
#    print(x)
#    pd.reset_option('display.max_rows')
#--------------------------------------------------------------------------------------
# Energy
#--------------------------------------------------------------------------------------
def get_energy():
    # Read Excel file, skip header and footer, skip first 2 columns and rename columns
    nm=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy = pd.read_excel('Energy Indicators.xls', skiprows=17,skip_footer =True,parse_cols="C:F", names=nm)
    energy.dropna(subset=['Energy Supply', 'Energy Supply per Capita', '% Renewable'], inplace=True) # remove footer

    #Convert Energy Supply to gigajoules (there are 1,000,000 gigajoules in a petajoule). 
    #For all countries which have missing data (e.g. data with "...") make sure this is reflected as np.NaN values.
    energy.replace("...", np.NaN, inplace=True)
    energy['Energy Supply']*=1000000

    # Remove digits from Country column & Parenthesis 
    energy['Country'] = energy['Country'].str.replace(r"\d","")
    energy['Country'] = energy['Country'].str.replace(r" \(.*\)","")

    # Rename the list of countries 
    # Replace multiple values in column
    dicts = {"Republic of Korea": "South Korea",
                 "United States of America": "United States",
                 "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                 "China, Hong Kong Special Administrative Region": "Hong Kong"}

    energy.replace({'Country':dicts}, inplace=True)
    #print_full(energy)
    return energy
#--------------------------------------------------------------------------------------
# GDP
#--------------------------------------------------------------------------------------
def get_GDP():
    #Next, load the GDP data from the file world_bank.csv
    GDP=pd.read_csv('world_bank.csv', header=4)
    
    #Rename Country column
    GDP.rename(columns={'Country Name':'Country'}, inplace=True)

    #rename some countries
    dict2={"Korea, Rep.": "South Korea","Iran, Islamic Rep.": "Iran","Hong Kong SAR, China": "Hong Kong"}
    GDP.replace({'Country':dict2}, inplace=True)
    return GDP
#--------------------------------------------------------------------------------------
# ScimEn
#--------------------------------------------------------------------------------------
def get_ScimEn():
    #Finally, load the Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology from the file scimagojr-3.xlsx,
    #which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame ScimEn.
    ScimEn=pd.read_excel('scimagojr-3.xlsx')
    return ScimEn

#--------------------------------------------------------------------------------------
# Join Datasets
#--------------------------------------------------------------------------------------
#Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). 
#Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).

# Get datasets
energy = get_energy()
GDP = get_GDP()
ScimEn = get_ScimEn()
    
#GDP last 10 years 
cols = ['Country'] + list(GDP.loc[:,'2006':]) 
GDPnew=GDP[cols]
    
# ScimEn top 15 
ScimEnNew=ScimEn.where(ScimEn['Rank'] <= 15).dropna()

def answer_one():
    # Merge DF - intersection
    a1 = pd.merge(pd.merge(energy, GDPnew, on='Country'), ScimEnNew, on='Country')
    #Set the index as Country and sort by alfabetical order
    a1 = a1.set_index('Country')
    a1 = a1.sort_values(by='Rank', ascending=True)
    
    #Correct column order
    cols=['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    a1=a1[cols]

    # Convert to correct type for grading - no warnings
    colsconv=['Self-citations','Citations','Citable documents','Documents','Rank','H index']
    a1[colsconv]=a1[colsconv].astype('int64')

    return a1

answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


In [2]:
#Check Data is clean
#print('{:15.15}{:15.15}{:15.15}'.format('dataframe', 'Starts with', 'Ends with'))
#print('{:15.15}{:15.15}{:15.15}'.format('----------', '-----------', '---------'))
#print('{:15.15}{:15.15}{:15.15}'.format('energy', get_energy()['Country'].iloc[0], get_energy()['Country'].iloc[-1]))
#print('{:15.15}{:15.15}{:15.15}'.format('GDP', get_GDP()['Country'].iloc[0], get_GDP()['Country'].iloc[-1]))
#print('{:15.15}{:15.15}{:15.15}'.format('ScimEn', get_ScimEn()['Country'].iloc[0], get_ScimEn()['Country'].iloc[-1]))

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [3]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>

In [4]:
# Merge union and intersection, NOTE: include all of ScimEn!
df_union = pd.merge(pd.merge(energy, GDPnew, on='Country', how='outer'), ScimEn, on='Country', how='outer')
df_inter = pd.merge(pd.merge(energy, GDPnew, on='Country'), ScimEn, on='Country')

#df_union.shape
#df_inter.shape

def answer_two():
    #To get the answer we need to subtract the intersection from the union
    return df_union.shape[0] - df_inter.shape[0]

answer_two()

156

<br>

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [5]:
def answer_three():
    Top15 = answer_one()
    #GDP last 10 years
    cols=list(GDP.loc[:,'2006':])
    #mean avg GDP
    avgGDP = Top15[cols].mean(axis=1)
    #Sorted in descending order
    avgGDP = avgGDP.sort_values(ascending=False) 
    return avgGDP
answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [6]:
def answer_four():
    Top15 = answer_one()
    #Add column of avg GDP from answer 3
    Top15['avgGDP']= answer_three()
    #Sort descending
    Top15.sort('avgGDP',ascending=False, inplace=True)
    #6th Largest average GDP
    largest6=Top15.iloc[5]
    
    #10 year span 
    return abs(largest6['2015']-largest6['2006'])
answer_four()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


246702696075.3999

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [7]:
def answer_five():
    Top15 = answer_one()
    #mean Energy supply per Capita
    return Top15['Energy Supply per Capita'].mean()
answer_five()

157.59999999999999

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [8]:
def answer_six():
    Top15 = answer_one()
    #Max %Renewable
    ct = Top15['% Renewable'].idxmax() #Return index of first occurrence of maximum over requested axis
    a6=(ct, Top15.loc[ct,:]['% Renewable']) #Tuple, country and % Renewable
    return a6
answer_six()

('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [9]:
def answer_seven():
    Top15 = answer_one()
    # Citation ratio
    Top15['Citation Ratio'] = Top15['Self-citations'] / Top15['Citations'] 
    Top15.reset_index()
    a7i=Top15['Citation Ratio'].idxmax() #Return index of first occurrence of maximum over requested axis
    a7=(a7i , Top15.loc[a7i,:]['Citation Ratio']) #Tuple, country and ratio
    return a7
answer_seven()

('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [10]:
def answer_eight():
    Top15 = answer_one()
    #Population estimate
    Top15['Population Estimate'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    #Sorted in Descending order
    Top15.sort_values(by='Population Estimate', inplace=True, ascending=False)
    #Reset index to get the third most populous
    a8=Top15.reset_index()
    return a8.iloc[2,0] # 3d most populous country
answer_eight()

'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [11]:
def answer_nine():
    Top15 = answer_one()
    #Population estimate
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    #Citable Documents per capita
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    #Correlation 
    a9=Top15[['Energy Supply per Capita', 'Citable docs per Capita']].corr().ix['Citable docs per Capita', 0]
    return a9
answer_nine()

0.79400104354429435

In [12]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [13]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [14]:
def answer_ten():
    Top15 = answer_one()
    #New column High Renew all 0s
    Top15['HighRenew'] = 0
    #What is the medium % Renewable
    mediantop15Renewablevalue= Top15['% Renewable'].median()
    #1 where %Renewable is >= to median
    Top15['HighRenew'][Top15['% Renewable'] >= mediantop15Renewablevalue] = 1
    return Top15['HighRenew']
answer_ten()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [15]:
def answer_eleven():
    Top15 = answer_one()
    d  = {'China':'Asia', 
                      'United States':'North America', 
                      'Japan':'Asia', 
                      'United Kingdom':'Europe', 
                      'Russian Federation':'Europe', 
                      'Canada':'North America', 
                      'Germany':'Europe', 
                      'India':'Asia',
                      'France':'Europe', 
                      'South Korea':'Asia', 
                      'Italy':'Europe', 
                      'Spain':'Europe', 
                      'Iran':'Asia',
                      'Australia':'Australia', 
                      'Brazil':'South America'}

    Continents=list(set(d.values())) # all continents
    Continents.sort() # Sort
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita'] #population estimate

    #Find Key where value = continent
    def find_key(input_dict, value):
        return {k for k, v in input_dict.items() if v == value}

    #New dataframe
    new_df = pd.DataFrame()

    #Loop through continents and populate new data frame 
    for i in range(0,len(Continents)):
        countries=find_key(d,Continents[i]) # Countries for that continent
        #Populate new df 
        new_df.set_value(i,'Continent',Continents[i])
        new_df.set_value(i,'size',len(countries))
        new_df.set_value(i,'sum',np.sum(Top15.loc[countries]['PopEst']))
        new_df.set_value(i,'mean',np.mean(Top15.loc[countries]['PopEst']))
        new_df.set_value(i,'std',np.std(Top15.loc[countries]['PopEst']))
    new_df=new_df.set_index('Continent')

    return new_df
answer_eleven()

,size,sum,mean,std
Continent,,,,
Asia,5.0,2.898666e+09,5.797333e+08,6.074036e+08
Australia,1.0,2.331602e+07,2.331602e+07,0.000000e+00
Europe,6.0,4.579297e+08,7.632161e+07,3.162885e+07
North America,2.0,3.528552e+08,1.764276e+08,1.411878e+08
South America,1.0,2.059153e+08,2.059153e+08,0.000000e+00


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [16]:
def answer_twelve():
    Top15 = answer_one()
    
    # Get new dict sorted by continent
    d  = {'China':'Asia', 
              'United States':'North America', 
              'Japan':'Asia', 
              'United Kingdom':'Europe', 
              'Russian Federation':'Europe', 
              'Canada':'North America', 
              'Germany':'Europe', 
              'India':'Asia',
              'France':'Europe', 
              'South Korea':'Asia', 
              'Italy':'Europe', 
              'Spain':'Europe', 
              'Iran':'Asia',
              'Australia':'Australia', 
              'Brazil':'South America'}

    Continents=list(set(d.values())) # all continents
    Continents.sort()

    #Find Key where value = continent
    def find_key(input_dict, value):
        return {k for k, v in input_dict.items() if v == value}

    new_tuple=[]
    #Loop through continents and populate new data frame 
    for i in range(0,len(Continents)):
        countries=find_key(d,Continents[i])
        temp=(Continents[i],countries)
        new_tuple.append(temp)
    
    #def func(item):
     #   if item in {'China', 'India', 'Iran', 'Japan', 'South Korea'}:
            #return 'Asia'
        #if item in {'Australia'}:
            #return 'Australia'
        #if item in {'France', 'Germany', 'Italy', 'Russian Federation', 'Spain', 'United Kingdom'}:
            #return 'Europe'
        #if item in {'Canada', 'United States'}:
            #return 'North America'
        #if item in {'Brazil'}:
            #return 'Brazil'   
        #else:
            #return None
    #Countries per Continent - Obtained from previous new_tuple
    def func(item):
        if item in new_tuple[0][1]:
            return new_tuple[0][0]
        if item in new_tuple[1][1]:
            return new_tuple[1][0]
        if item in new_tuple[2][1]:
            return new_tuple[2][0]
        if item in new_tuple[3][1]:
            return new_tuple[3][0]
        if item in new_tuple[4][1]:
            return new_tuple[4][0]   
        else:
            return None
                         
    return Top15.groupby([func,pd.cut(Top15['% Renewable'],5)]).size()
answer_twelve()

               % Renewable     
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [17]:
def answer_thirteen():
    Top15 = answer_one()
    #Population estimate as float
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita'].astype(float)
    #Format with ,
    return Top15['PopEst'].map('{0:,}'.format)
answer_thirteen()

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: PopEst, dtype: object

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [18]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [19]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!